# Technical solution to space debris collection

Here, we formulate the problem to solve to optimize space cleaning. 

The problem of trash in space has become a boiling issue: objects are orbiting around the earth, endangering the space expeditions. They are composed of dysfunctional instruments that are no longer in use, or broken parts that were torn apart from a larger device. In spite of being mostly very small, they travel at such impressive speeds that by Newton's law, any shock with one of these could cause tremendous damage to a space ship or a satellite. Since these space expeditions are very costly, it is a major challenge to avoid them, and to remove them from space in the long term. Therefore, the challenge of cleaning space has been undertaken by companies, space agencies and public institutions.

To optimize their travel and collection schedule, two main constraints should be fulfilled:
1) Since the space ship has limited capacity, it can only collect so much that it can contain.
2) Fuel can also be contained in limited amounts, and is in itself very expensive. It is hence desirable to travel the longest distance that the fuel quantity allows.

To formulate it simply, the challenge is to find the shortest path between debris that gathers the most of them, within the capactity of the ship. Using brute force, one would have to try all possible combinations of debris to find the one that fulfills these conditions the best. It is therefore a combinatorial optimization problem that would grow exponentially in the number of debris to choose from. It is therefore unscalable and untractable to classical computer at large scales. To circumvent this delicate issue, we propose to use a quantum-classical algorithm to get a better solution than a classical computer could get. Using the gate-based model in quantum computing, we solve the problem with the Quantum Approximate Optimization Algorithm.

## Cost function 

To perform the optimization, we first express the problem in a cost function that we attempt to minimize. We start with a set of N debris. We encode the information about the debris in a bitstring:\begin{equation}
z = z_1 z_2 \cdots z_N, \text{where } z_i = \begin{cases}
1, & \text{if debris i is collected} \\
0, & \text{if otherwise.}
\end{cases}
\end{equation}

To solve the problem with brute force method, we must inspect all possible bitstrings $z$ and pick the one that maximizes the volume of the debris collected. Therefore, it is a combinatorial optimization problem. To formulate it, we pair each debris $i$ with its weight/size $w_i$. The debris locations can be reported on a graph with edge set $E$ and vertex set $V$ such that $G = \{E,V\}$. Each vertex is associated with a weight, and each edge $i -j$ with a distance $d_{ij}$. With these notations, we encode the problem into the cost function:
\begin{equation}
C(z) = (V-\sum_{i=1}^{N} w_i z_i)^2 + \sum_{i<j}^{N} d_{ij} z_i z_j
\end{equation}
where the first term corresponds to the knapsack problem and the second aims at minimizing the travelled distance. To simplify the implementation, the constant term can be removed:
\begin{align}
C(z) &=  (-2V\sum_{i=1}^{N} w_i z_i + 2\sum_{i<j}^N w_i w_j z_i z_j) + \sum_{i<j}^{N} d_{ij} z_i z_j \\
 & = -2V \sum_{i=1}^{N} w_i z_i + \sum_{i<j}^N (2 w_i w_j + d_{ij})\, z_i z_j
\end{align}

## Mapping to a Hamiltonian

The next step is to transform the bits $z_i$ into quantum operators. To preserve the mapping to 0 and 1, one can perform the following mapping:
\begin{equation}
z_i \longrightarrow \frac{1}{2}(\mathbb{1}_i - Z_i)
\end{equation}
where Z_i is the Pauli Z operator acting on qubit $i$. By simply inserting this ansatz into the cost function, we can construct the problem Hamiltonian:
\begin{align}
\hat{H}_P &=  -2V \sum_{i=1}^{N} w_i \frac{1}{2}(\mathbb{1}_i - Z_i) + \sum_{i<j}^N (2 w_i w_j + d_{ij})\, \frac{1}{4}(\mathbb{1}_i - Z_i) (\mathbb{1}_i - Z_j) \\
&= -2V \sum_{i=1}^{N} w_i \frac{1}{2}(\mathbb{1}_i - Z_i) + \sum_{i<j}^N (2 w_i w_j + d_{ij})\, \frac{1}{4}(\mathbb{1}_i \mathbb{1}_j - Z_i - Z_j + Z_iZ_j). 
\end{align}
Since the terms acting with the identity only shift all energy levels irrespective of the states, we can omit them in the formulation. The final Hamiltonian is therefore:
\begin{align}
\hat{H}_P &= \sum_{i=1}^{N} Vw_i Z_i + \sum_{i<j} (2 w_i w_j + d_{ij})\, \frac{1}{4}( - Z_i - Z_j + Z_iZ_j)\\
&= \sum_{i=1}^{N} h_i Z_i + \sum_{i<j}^N g_{ij} ( - Z_i - Z_j + Z_iZ_j)\\
&= \sum_{i=1}^{N} (h_i - \sum_{j>i} g_{ij} -\sum_{j<i} g_{ij})\, Z_i + \sum_{i<j} g_{ij} Z_iZ_j\\
&= \sum_{i=1}^{N} (h_i - \sum_{j \not= i} g_{ij}) \, Z_i + \sum_{i<j} g_{ij} Z_iZ_j,
\end{align}
where we have defined $h_i = Vw_i$ and $g_{ij} = \frac{1}{4} (2w_i w_j+d_{ij}) $

## QAOA algorithm

With the problem Hamiltonian, we are now ready to implement the QAOA algorithm to find the optimal trips. The steps are the following:
1) Prepare a qubit register with N qubits in the computational ground state $|0\rangle^{\otimes N}$;
2) Apply the Hadamard gate on all of them, yielding an equal superposition of all states in the computational basis, or alternatively of all possible combinations of debris.
3) Apply the QAOA Ansatz for a given number of layers p. For each p:
    - evolve the N-qubit state with the problem Hamiltonian above, resulting in the evolution unitary: $U_P=e^{i \gamma_p H_P}$
    - evolve the N-qubit state with the mixer Hamiltonian: $U_M = e^{i\beta_p\sum_{i=1}^N X_i}$
The resulting state is:\begin{equation}
|\gamma, \beta\rangle = U_M^{(p)} U_P^{(p)} \cdots U_M^{(1)} U_P^{(1)} \frac{1}{\sqrt{2^N}} \sum_{z\in\{0,1\}^{\otimes N}} |z\rangle
\end{equation}
4) Calculate the expectation value of the problem Hamiltonian with respect to the variational state above: \begin{equation}\langle \gamma, \beta| H_P |\gamma, \beta\rangle\end{equation}

5) Use a classical optimizer to select the $\gamma$ and $\beta$ that minimize this expectation value.
6) Run the QAOA circuit one more time and select the most probable basis state: it contains the answer to the problem.